## Installation
Plotly and cufflinks are used for visualization.

In [1]:
# !pip install plotly pandas

## Imports

In [2]:
import sys
sys.path.append('..')
from src.finance_ml.backtests.strategy import Strategy, MarketAction, Operator
import warnings
warnings.filterwarnings('ignore')


## Loading Data


In [3]:
from src.finance_ml.data_preparation.data_preparation import DataLoader
from src.finance_ml.indicators.indicators import Indicators

dataloader = DataLoader(time_index_col= 'DATE', 
                    keep_cols = ['VOLUME','OPEN', 'HIGHT', 'LOW', 'CLOSE', 'VW','TRANSACTIONS'])




In [4]:
# Example Files
fname_RUBEUR = 'FX/RUBEUR_2020-04-07_2022-04-06.parquet'
fname_USDBRL = 'FX/USDBRL_2020-04-07_2022-04-06.parquet'
fname_AAPL = 'equities/AAPL_2020-04-07_2022-04-06.parquet'

# No. of Records from example dataset
N =-1

In [5]:
# Dataset chosen in this simulation
ticker = 'AAPL'
fname = fname_AAPL

In [6]:
# loading assets into to an unique df
df = dataloader.load_dataset({ticker:'../data/'+fname}).iloc[:N]
display(df)

,AAPL_VOLUME,AAPL_OPEN,AAPL_HIGHT,AAPL_LOW,AAPL_CLOSE,AAPL_VW,AAPL_TRANSACTIONS
DATE,,,,,,,
2020-04-07 12:17:00,14256.0,67.3500,67.3625,67.3275,67.3375,67.3454,45
2020-04-07 12:18:00,16552.0,67.3501,67.3625,67.3250,67.3250,67.3407,57
2020-04-07 12:19:00,80172.0,67.3250,67.4475,67.3250,67.4225,67.3937,149
2020-04-07 12:20:00,183740.0,67.4475,67.4700,67.3125,67.3700,67.3671,159
2020-04-07 12:21:00,35372.0,67.3500,67.3850,67.2975,67.2975,67.3469,102
...,...,...,...,...,...,...,...
2022-04-06 23:54:00,3183.0,171.0100,171.0100,170.9900,170.9900,171.0019,104
2022-04-06 23:55:00,840.0,170.9800,170.9800,170.9300,170.9700,170.9593,26
2022-04-06 23:56:00,2816.0,170.9300,170.9500,170.8500,170.9500,170.8997,56


## Strategy

Our first and simple strategy:
> What if We buy when price decreased from yesterday and sell when price increased from yesterday?

In [7]:
tdata = df.copy()
tdata.columns = [c.split('_')[-1] for c in tdata.columns]
tdata = tdata.reset_index()
tdata.head()

# make daily data
tdata = tdata.set_index('DATE').resample('D')
tdata = tdata.aggregate({'OPEN': 'first', 'HIGHT': 'max', 'LOW': 'min', 'CLOSE': 'last', 'VOLUME': 'sum', 'VW': 'mean', 'TRANSACTIONS': 'sum'})
tdata = tdata.dropna()
tdata = tdata.reset_index()

### Short Flow of Our Strategy Class
1. Create Object
2. Load Data
3. Add Rule
4. Simulate
5. Check Trade Summary
6. Plot Trade History

#### Object of a Class

In [8]:
strategy = Strategy(name="Simple Strategy", cash=1000, commission=0.05, min_positions=1)
strategy, type(strategy)

(Strategy: Simple Strategy, src.finance_ml.backtests.strategy.Strategy)

#### Load Data into Strategy

In [9]:
data = tdata.copy()
strategy.load_data(tdata, date_column='DATE')
strategy.data.head()

,DATE,OPEN,HIGHT,LOW,CLOSE,VOLUME,VW,TRANSACTIONS
0,2020-04-07,67.3500,67.9250,64.2414,65.0000,192850144.0,65.876864,459908
1,2020-04-08,65.0000,66.8425,64.3550,66.8400,154378980.0,66.041467,390477
2,2020-04-09,67.1375,67.5750,65.5888,67.2875,155875940.0,66.782556,414970
3,2020-04-13,66.3750,68.7375,66.1250,68.6625,125174624.0,67.437177,342534
4,2020-04-14,69.3750,72.0625,69.0775,71.4750,182234064.0,71.011849,476274


#### Add a Single Rule
Follow the docstring for more info. 

Let's add a buy rule. We buy when the yesterday's close price is greater than today's. That is when price dropped.

In [10]:
rule1 = strategy.add_single_rule('CLOSE[N-2]>CLOSE[N-1]', action=MarketAction.BUY, action_quantity='ALL')
rule1

Rule(id='Simple Strategy_rule_1', condition='CLOSE[N-2]>CLOSE[N-1]', operator='&', action=<MarketAction.BUY: 'BUY'>, action_quantity='ALL', equation="self._curr_data['CLOSE'].iloc[-2] > self._curr_data['CLOSE'].iloc[-1]")

The above rule will be converted into Python `eval` runnable code like below. (But after some refining!)

In [11]:
rule1.equation

"self._curr_data['CLOSE'].iloc[-2] > self._curr_data['CLOSE'].iloc[-1]"

Let's add another rule for selling.

In [12]:
strategy.add_single_rule('CLOSE[N-2]<CLOSE[N-1]', action=MarketAction.SELL, action_quantity='ALL')

Rule(id='Simple Strategy_rule_2', condition='CLOSE[N-2]<CLOSE[N-1]', operator='&', action=<MarketAction.SELL: 'SELL'>, action_quantity='ALL', equation="self._curr_data['CLOSE'].iloc[-2] < self._curr_data['CLOSE'].iloc[-1]")

Let's see all rules.

In [13]:
strategy._raw_rules

{<MarketAction.BUY: 'BUY'>: [Rule(id='Simple Strategy_rule_1', condition='CLOSE[N-2]>CLOSE[N-1]', operator='&', action=<MarketAction.BUY: 'BUY'>, action_quantity='ALL', equation="self._curr_data['CLOSE'].iloc[-2] > self._curr_data['CLOSE'].iloc[-1]")],
 <MarketAction.SELL: 'SELL'>: [Rule(id='Simple Strategy_rule_2', condition='CLOSE[N-2]<CLOSE[N-1]', operator='&', action=<MarketAction.SELL: 'SELL'>, action_quantity='ALL', equation="self._curr_data['CLOSE'].iloc[-2] < self._curr_data['CLOSE'].iloc[-1]")]}

#### Run the simulation

In [14]:
strategy.simulate()

#### Check the summary

In [15]:
strategy.summary()

Strategy Name: Simple Strategy
First Trade Date: 2020-04-15 00:00:00
Last Trade Date: 2022-04-05 00:00:00
Total Trades: 275
Final PnL: 554.539574752519
Starting Cash: 1000
Final Cash: 0.00012693744724856515
Final Positions: 8.883082558943267
Final Portfolio Value: 1554.539574752519
Total Commission: 163.93082243385268
Total Buy Commission: 82.18748828089994
Total Sell Commission: 81.74333415295277
Total Buy Amount: 208969.58834768005
Total Sell Amount: 207969.58847461749
Total Buy Quantity: 1643.7497656179985
Total Sell Quantity: 1634.8666830590553
Total Buy Trades: 138
Total Sell Trades: 137


We actually made some money.

#### History of Trades

In [16]:
strategy.history_df

,date,action,quantity,close_price,commission,amount,positions,cash,portfolio_value
0,2020-04-15,BUY,14.124287,70.7500,0.706214,999.999501,14.124287,0.000499,999.293786
1,2020-04-16,SELL,14.124287,73.8425,0.706214,1042.266429,0.000000,1042.266928,1042.266928
2,2020-04-17,BUY,14.761935,70.5550,0.738097,1042.266405,14.761935,0.000523,1041.528831
3,2020-04-22,SELL,14.761935,68.5625,0.738097,1011.377056,0.000000,1011.377580,1011.377580
4,2020-04-23,BUY,14.802445,68.2750,0.740122,1011.377037,14.802445,0.000542,1010.637457
...,...,...,...,...,...,...,...,...,...
270,2022-03-10,BUY,10.021882,158.3600,0.501094,1587.566320,10.021882,0.000158,1587.065384
271,2022-03-15,SELL,10.021882,155.0000,0.501094,1552.890609,0.000000,1552.890767,1552.890767
272,2022-03-30,BUY,8.720675,178.0200,0.436034,1552.890644,8.720675,0.000123,1552.454733
273,2022-04-04,SELL,8.720675,178.3600,0.436034,1554.983606,0.000000,1554.983729,1554.983729


#### Plots
Let's see the history on a graph.

In [17]:
strategy.plot_history(title="Simple Strategy", yaxis_title="Stock's Price in USD", xaxis_title="Date")

### What if we sell when SMA 10 crosses over SMA 30?


Let's use indicators from our package.

In [18]:
from src.finance_ml.indicators.indicators import Indicators

In [19]:
# Process data normalization
norm_data = True

# Select Indicator transformer
ind_processor = Indicators(ticker = ticker, norm_data = False, calc_all=False, list_ind=['SMA'], SMA_win=10)

# Calculate Indicators over input dataframe
df = ind_processor.fit_transform(df)
df

,AAPL_VOLUME,AAPL_OPEN,AAPL_HIGHT,AAPL_LOW,AAPL_CLOSE,AAPL_VW,AAPL_TRANSACTIONS,AAPL_SMA_w10
DATE,,,,,,,,
2020-04-07 12:17:00,14256.0,67.3500,67.3625,67.3275,67.3375,67.3454,45,NaN
2020-04-07 12:18:00,16552.0,67.3501,67.3625,67.3250,67.3250,67.3407,57,NaN
2020-04-07 12:19:00,80172.0,67.3250,67.4475,67.3250,67.4225,67.3937,149,NaN
2020-04-07 12:20:00,183740.0,67.4475,67.4700,67.3125,67.3700,67.3671,159,NaN
2020-04-07 12:21:00,35372.0,67.3500,67.3850,67.2975,67.2975,67.3469,102,NaN
...,...,...,...,...,...,...,...,...
2022-04-06 23:54:00,3183.0,171.0100,171.0100,170.9900,170.9900,171.0019,104,171.142
2022-04-06 23:55:00,840.0,170.9800,170.9800,170.9300,170.9700,170.9593,26,171.119
2022-04-06 23:56:00,2816.0,170.9300,170.9500,170.8500,170.9500,170.8997,56,171.094


#### Using TA for Indicators

In [20]:
tdata.head()

,DATE,OPEN,HIGHT,LOW,CLOSE,VOLUME,VW,TRANSACTIONS
0,2020-04-07,67.3500,67.9250,64.2414,65.0000,192850144.0,65.876864,459908
1,2020-04-08,65.0000,66.8425,64.3550,66.8400,154378980.0,66.041467,390477
2,2020-04-09,67.1375,67.5750,65.5888,67.2875,155875940.0,66.782556,414970
3,2020-04-13,66.3750,68.7375,66.1250,68.6625,125174624.0,67.437177,342534
4,2020-04-14,69.3750,72.0625,69.0775,71.4750,182234064.0,71.011849,476274


In [21]:
from ta.trend import SMAIndicator

sma_10 = SMAIndicator(close=tdata['CLOSE'], window=10, fillna=False)
tdata['SMA_10'] = sma_10.sma_indicator()
sma_30 = SMAIndicator(close=tdata['CLOSE'], window=30, fillna=False)
tdata['SMA_30'] = sma_30.sma_indicator()

tdata

,DATE,OPEN,HIGHT,LOW,CLOSE,VOLUME,VW,TRANSACTIONS,SMA_10,SMA_30
0,2020-04-07,67.3500,67.9250,64.2414,65.0000,192850144.0,65.876864,459908,NaN,NaN
1,2020-04-08,65.0000,66.8425,64.3550,66.8400,154378980.0,66.041467,390477,NaN,NaN
2,2020-04-09,67.1375,67.5750,65.5888,67.2875,155875940.0,66.782556,414970,NaN,NaN
3,2020-04-13,66.3750,68.7375,66.1250,68.6625,125174624.0,67.437177,342534,NaN,NaN
4,2020-04-14,69.3750,72.0625,69.0775,71.4750,182234064.0,71.011849,476274,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
536,2022-03-31,178.4000,179.0100,174.4000,174.8600,86829829.0,176.530800,714804,172.490,164.410667
537,2022-04-01,175.2500,175.8800,171.9400,174.0000,72207239.0,173.765653,659138,173.456,164.866667
538,2022-04-04,174.1800,178.5500,173.8000,178.3600,70770091.0,176.936615,628283,174.792,165.446000
539,2022-04-05,178.6600,178.6600,174.4150,175.0000,68992759.0,176.312737,629317,175.397,165.799333


#### Simulation Results

In [22]:
sma_strategy = Strategy(name="SMA Strategy", cash=1000, commission=0.05, min_positions=1)
data = tdata.copy()
sma_strategy.load_data(tdata, date_column='DATE')
sma_strategy.add_single_rule('SMA_10[N-1]>SMA_30[N-1]', action=MarketAction.SELL, action_quantity='ALL')
sma_strategy.add_single_rule('SMA_10[N-1]<SMA_30[N-1]', action=MarketAction.BUY, action_quantity='ALL')
sma_strategy.simulate()

sma_strategy.summary()
sma_strategy.plot_history(title="SMA Strategy", yaxis_title="Stock's Price in USD", xaxis_title="Date", plot_columns=['CLOSE', 'SMA_10', 'SMA_30'])

Strategy Name: SMA Strategy
First Trade Date: 2020-09-16 00:00:00
Last Trade Date: 2022-03-25 00:00:00
Total Trades: 16
Final PnL: 447.4010327081942
Starting Cash: 1000
Final Cash: 1447.4010327081942
Final Positions: 0.0
Final Portfolio Value: 1447.4010327081942
Total Commission: 7.311660314609401
Total Buy Commission: 3.6558301573047
Total Sell Commission: 3.6558301573047
Total Buy Amount: 9826.59086486464
Total Sell Amount: 10273.991897572834
Total Buy Quantity: 73.11660314609401
Total Sell Quantity: 73.11660314609401
Total Buy Trades: 8
Total Sell Trades: 8


### Buy assets worth 20% of available cash when price decreased but sell 30% of assets when increased.

In [23]:
per_strategy = Strategy(name="SMA Strategy", cash=1000, commission=0.05, min_positions=0.5)
data = tdata.copy()
per_strategy.load_data(tdata, date_column='DATE')
per_strategy.add_single_rule('CLOSE[N-1]>CLOSE[N-2]', action=MarketAction.SELL, action_quantity='30%')
per_strategy.add_single_rule('CLOSE[N-1]<CLOSE[N-2]', action=MarketAction.BUY, action_quantity='20%')
per_strategy.simulate()

per_strategy.summary()
per_strategy.plot_history(title="Simple Strategy", yaxis_title="Stock's Price in USD", xaxis_title="Date")

Strategy Name: SMA Strategy
First Trade Date: 2020-04-15 00:00:00
Last Trade Date: 2022-04-05 00:00:00
Total Trades: 496
Final PnL: 487.6512834363323
Starting Cash: 1000
Final Cash: 724.5876184474748
Final Positions: 4.360363799936328
Final Portfolio Value: 1487.6512834363323
Total Commission: 31.542038979075627
Total Buy Commission: 15.880028584536221
Total Sell Commission: 15.662010394539404
Total Buy Amount: 40830.9017941636
Total Sell Amount: 40555.489412611074
Total Buy Quantity: 317.60057169072445
Total Sell Quantity: 313.2402078907881
Total Buy Trades: 244
Total Sell Trades: 252


### Trading Based on PPO (Percentage Price Oscillator)
(From Investopedia)
* This is a momentum indicator (determines the strength or weakness of a value). But we can view the volatility too.
* Two EMAs, 26 period and 12 periods are used to calculate PPO.
* It contains 2 lines, PPO line and signal line. Signal line is an EMA of the 9 Period PPO, so it moves slower than PPO.
* When PPO line crosses the signal line, it is the time for rise/fall of the price or stock.
* When PPO line crosses over the signal line from below, then it is a buy signal. Reversely, it is a sell signal when PPO line crosses below the signal line from above.

$$
PPO = \frac{\text{12 Period EMA - 26 Period EMA}}{\text{26 Period EMA}} * 100 \\\
signal = \text{9 period EMA of PPO} \\\
PPO histogram = \text{PPO - Signal Line}
$$

In [24]:
from ta.momentum import PercentagePriceOscillator

ppo = PercentagePriceOscillator(close=tdata['CLOSE'], window_slow=26, window_fast=12, window_sign=9, fillna=False)

tdata['PPO'] = ppo.ppo()
tdata['PPO_Signal'] = ppo.ppo_signal()
tdata['PPO_Hist'] = ppo.ppo_hist()
tdata

,DATE,OPEN,HIGHT,LOW,CLOSE,VOLUME,VW,TRANSACTIONS,SMA_10,SMA_30,PPO,PPO_Signal,PPO_Hist
0,2020-04-07,67.3500,67.9250,64.2414,65.0000,192850144.0,65.876864,459908,NaN,NaN,NaN,NaN,NaN
1,2020-04-08,65.0000,66.8425,64.3550,66.8400,154378980.0,66.041467,390477,NaN,NaN,NaN,NaN,NaN
2,2020-04-09,67.1375,67.5750,65.5888,67.2875,155875940.0,66.782556,414970,NaN,NaN,NaN,NaN,NaN
3,2020-04-13,66.3750,68.7375,66.1250,68.6625,125174624.0,67.437177,342534,NaN,NaN,NaN,NaN,NaN
4,2020-04-14,69.3750,72.0625,69.0775,71.4750,182234064.0,71.011849,476274,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
536,2022-03-31,178.4000,179.0100,174.4000,174.8600,86829829.0,176.530800,714804,172.490,164.410667,2.067174,0.910514,1.156660
537,2022-04-01,175.2500,175.8800,171.9400,174.0000,72207239.0,173.765653,659138,173.456,164.866667,2.035360,1.135483,0.899877
538,2022-04-04,174.1800,178.5500,173.8000,178.3600,70770091.0,176.936615,628283,174.792,165.446000,2.188580,1.346102,0.842478
539,2022-04-05,178.6600,178.6600,174.4150,175.0000,68992759.0,176.312737,629317,175.397,165.799333,2.126622,1.502206,0.624416


In [25]:
import plotly.graph_objects as go

ppo_strategy = Strategy(name="PPO Strategy", cash=1000, commission=0.05, min_positions=1)
ppo_strategy.load_data(tdata, date_column='DATE')
ppo_strategy.add_single_rule('PPO[N-1]>PPO_Signal[N-1]', action=MarketAction.BUY, action_quantity='ALL')
ppo_strategy.add_single_rule('PPO[N-1]<PPO_Signal[N-1]', action=MarketAction.SELL, action_quantity='ALL')
ppo_strategy.simulate()

ppo_strategy.summary()
fig = ppo_strategy.plot_history(title="Simple Strategy", yaxis_title="Stock's Price in USD", xaxis_title="Date", plot_columns=['CLOSE', 'PPO', 'PPO_Signal'])
fig.add_trace(go.Bar(x=tdata.DATE, y=tdata.PPO_Hist, name="PPO_Hist", marker=dict(color=tdata["PPO_Hist"].apply(
                        lambda x: "green" if x>0 else "red"
                    ))),secondary_y=True)

Strategy Name: PPO Strategy
First Trade Date: 2020-06-09 00:00:00
Last Trade Date: 2022-03-18 00:00:00
Total Trades: 33
Final PnL: 394.7447966522486
Starting Cash: 1000
Final Cash: 0.00012914580543110787
Final Positions: 8.486945767959371
Final Portfolio Value: 1394.7447966522486
Total Commission: 15.30553134213137
Total Buy Commission: 7.86493931526467
Total Sell Commission: 7.440592026866701
Total Buy Amount: 20639.637849278784
Total Sell Amount: 19639.637978424595
Total Buy Quantity: 157.2987863052934
Total Sell Quantity: 148.811840537334
Total Buy Trades: 17
Total Sell Trades: 16


Note:
- Hiding the portfolio value above to see the histogram, we can see that we were buying just when price was about to go up and sell when it was going down.
- Showing only PPO, PPO_Signal and PPO_Hist, we can see the crossovers too.



### Using Multiple Indicators
We have used two different indicators above, Simple Moving Average and PPO. We also so their performance separately but can we combine them both?
Yes and let's see how.

In [26]:
tdata.head()

,DATE,OPEN,HIGHT,LOW,CLOSE,VOLUME,VW,TRANSACTIONS,SMA_10,SMA_30,PPO,PPO_Signal,PPO_Hist
0,2020-04-07,67.3500,67.9250,64.2414,65.0000,192850144.0,65.876864,459908,NaN,NaN,NaN,NaN,NaN
1,2020-04-08,65.0000,66.8425,64.3550,66.8400,154378980.0,66.041467,390477,NaN,NaN,NaN,NaN,NaN
2,2020-04-09,67.1375,67.5750,65.5888,67.2875,155875940.0,66.782556,414970,NaN,NaN,NaN,NaN,NaN
3,2020-04-13,66.3750,68.7375,66.1250,68.6625,125174624.0,67.437177,342534,NaN,NaN,NaN,NaN,NaN
4,2020-04-14,69.3750,72.0625,69.0775,71.4750,182234064.0,71.011849,476274,NaN,NaN,NaN,NaN,NaN


In [27]:
complex_strategy = Strategy(name="Complex Strategy", cash=1000, commission=0.05, min_positions=1)
complex_strategy.load_data(tdata, date_column='DATE')
complex_strategy.add_single_rule('SMA_10[N-1]>SMA_30[N-1]', action=MarketAction.SELL, action_quantity='ALL', operator=Operator.OR)
complex_strategy.add_single_rule('SMA_10[N-1]<SMA_30[N-1]', action=MarketAction.BUY, action_quantity='ALL', operator=Operator.OR)
complex_strategy.add_single_rule('PPO[N-1]>PPO_Signal[N-1]', action=MarketAction.BUY, action_quantity='ALL')
complex_strategy.add_single_rule('PPO[N-1]<PPO_Signal[N-1]', action=MarketAction.SELL, action_quantity='ALL')

complex_strategy.simulate()
complex_strategy.summary()
fig = complex_strategy.plot_history(title="Complex Strategy", yaxis_title="Stock's Price in USD", xaxis_title="Date")
fig.show()

Strategy Name: Complex Strategy
First Trade Date: 2020-06-09 00:00:00
Last Trade Date: 2022-02-23 00:00:00
Total Trades: 33
Final PnL: 487.39008688701983
Starting Cash: 1000
Final Cash: 0.00014471878512267722
Final Positions: 9.277631874801864
Final Portfolio Value: 1487.3900868870198
Total Commission: 16.47363935752944
Total Buy Commission: 8.468760475634765
Total Sell Commission: 8.004878881894673
Total Buy Amount: 22216.142241959424
Total Sell Amount: 21216.142386678206
Total Buy Quantity: 169.3752095126953
Total Sell Quantity: 160.09757763789344
Total Buy Trades: 17
Total Sell Trades: 16


In [28]:
complex_strategy._eval_rules

{<MarketAction.BUY: 'BUY'>: "(self._curr_data['SMA_10'].iloc[-1] < self._curr_data['SMA_30'].iloc[-1]) | (self._curr_data['PPO'].iloc[-1] > self._curr_data['PPO_Signal'].iloc[-1])",
 <MarketAction.SELL: 'SELL'>: "(self._curr_data['SMA_10'].iloc[-1] > self._curr_data['SMA_30'].iloc[-1]) | (self._curr_data['PPO'].iloc[-1] < self._curr_data['PPO_Signal'].iloc[-1])"}

### What if we buy with whatever we have but sell only 50% of assets?

In [29]:
complex_strategy = Strategy(name="Complex Strategy", cash=1000, commission=0.05, min_positions=0.1)
complex_strategy.load_data(tdata, date_column='DATE')
complex_strategy.add_single_rule('SMA_10[N-1]>SMA_30[N-1]', action=MarketAction.SELL, action_quantity='ALL', operator=Operator.OR)
complex_strategy.add_single_rule('SMA_10[N-1]<SMA_30[N-1]', action=MarketAction.BUY, action_quantity='ALL', operator=Operator.OR)
complex_strategy.add_single_rule('PPO[N-1]>PPO_Signal[N-1]', action=MarketAction.BUY, action_quantity='ALL')
complex_strategy.add_single_rule('PPO[N-1]<PPO_Signal[N-1]', action=MarketAction.SELL, action_quantity='50%')

complex_strategy.simulate()
complex_strategy.summary()
fig = complex_strategy.plot_history(title="Complex Strategy", yaxis_title="Stock's Price in USD", xaxis_title="Date", plot_columns=['CLOSE', 'SMA_10', 'SMA_30', 'PPO', 'PPO_Signal'])
fig.show()

Strategy Name: Complex Strategy
First Trade Date: 2020-06-09 00:00:00
Last Trade Date: 2022-02-23 00:00:00
Total Trades: 104
Final PnL: 612.2724887693066
Starting Cash: 1000
Final Cash: 0.0001545432301099936
Final Positions: 10.056588911090797
Final Portfolio Value: 1612.2724887693066
Total Commission: 16.052343232892593
Total Buy Commission: 8.277586339223566
Total Sell Commission: 7.774756893669026
Total Buy Amount: 21717.58856929127
Total Sell Amount: 20717.588723834502
Total Buy Quantity: 165.55172678447133
Total Sell Quantity: 155.49513787338054
Total Buy Trades: 17
Total Sell Trades: 87


### What if we started with 10000 USD in cash?

In [30]:
complex_strategy = Strategy(name="Complex Strategy", cash=10000, commission=0.05, min_positions=0.1)
complex_strategy.load_data(tdata, date_column='DATE')
complex_strategy.add_single_rule('SMA_10[N-1]>SMA_30[N-1]', action=MarketAction.SELL, action_quantity='ALL', operator=Operator.OR)
complex_strategy.add_single_rule('SMA_10[N-1]<SMA_30[N-1]', action=MarketAction.BUY, action_quantity='ALL', operator=Operator.OR)
complex_strategy.add_single_rule('PPO[N-1]>PPO_Signal[N-1]', action=MarketAction.BUY, action_quantity='10%')
complex_strategy.add_single_rule('PPO[N-1]<PPO_Signal[N-1]', action=MarketAction.SELL, action_quantity='50%')

complex_strategy.simulate()
complex_strategy.summary()
fig = complex_strategy.plot_history(title="Complex Strategy", yaxis_title="Stock's Price in USD", xaxis_title="Date", plot_columns=['CLOSE', 'SMA_10', 'SMA_30', 'PPO', 'PPO_Signal'])
fig.show()

Strategy Name: Complex Strategy
First Trade Date: 2020-06-09 00:00:00
Last Trade Date: 2022-04-05 00:00:00
Total Trades: 440
Final PnL: 4420.029019344351
Starting Cash: 10000
Final Cash: 413.190857963662
Final Positions: 80.03907520788965
Final Portfolio Value: 14420.02901934435
Total Commission: 106.38790938525426
Total Buy Commission: 55.19493157282437
Total Sell Commission: 51.19297781242989
Total Buy Amount: 144961.09982927458
Total Sell Amount: 135374.29068723822
Total Buy Quantity: 1103.8986314564872
Total Sell Quantity: 1023.8595562485978
Total Buy Trades: 331
Total Sell Trades: 109
